In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../Code/utils')

In [2]:
import objects

# Generating total combinations of year/month/tribunal

In [3]:
years = objects.YEARS
months = objects.MONTHS
tribunals = objects.TRIBUNALS

In [4]:
df = pd.DataFrame(columns=['file', 'year', 'month', 'tribunal'])

In [5]:
for year in years:
    for month in months:
        for tribunal in tribunals:
            row = ['master_'+str(year)+'_'+month+'_'+tribunal+'.csv', year, month, tribunal]
            df.loc[len(df)] = row

# Loading file with total number of cases per year/month/tribunal

In [6]:
file_total = '../Outputs/master/total_cases.csv'
df_total = pd.read_csv(file_total, encoding='latin1')

In [7]:
df_total['file'] = 'master_'+df_total['Year'].astype(str)+'_'+df_total['Month']+'_'+df_total['Tribunal']+'.csv'

In [8]:
df2 = df.merge(df_total[['file', 'Total cases', 'Number of pages']], on='file', indicator=True, how='outer')

In [9]:
df_total.columns

Index(['Year', 'Month', 'Tribunal', 'Total cases', 'Number of pages', 'file'], dtype='object')

In [10]:
df2['_merge'].value_counts()

both          744
left_only       0
right_only      0
Name: _merge, dtype: int64

In [11]:
df2['in_total_cases'] = 0
df2.loc[df2['_merge']=='both', 'in_total_cases'] = 1

# Loading total number of cases scraped in master files

In [12]:
from os import walk

In [13]:
for _, _, f3 in walk('../Outputs/master'):
    files = f3

In [14]:
master_files = []
for file in files:
    if file[:6] == 'master':
        master_files.append(file)

In [15]:
df2['number_cases_obtained'] = 0
for file in master_files:
    df_temp = pd.read_csv('../Outputs/master/'+file, encoding='latin1')
    df2.loc[df2['file']==file, 'number_cases_obtained'] = len(df_temp)

# Tagging finished cases

In [16]:
df2['percentage_finished'] = 0
df2.loc[df2['in_total_cases']==1, 'percentage_finished'] = df2['number_cases_obtained'] / df2['Total cases']

In [17]:
df2['finished'] = 0
df2.loc[df2['percentage_finished']>=1, 'finished'] = 1

In [18]:
len(df2)

744

In [19]:
df2['finished'].value_counts()

1    744
Name: finished, dtype: int64

In [20]:
len(df2[df2['percentage_finished']>1])

0

In [21]:
df2[(df2['finished']==0) | (df2['percentage_finished']>1)]

,file,year,month,tribunal,Total cases,Number of pages,_merge,in_total_cases,number_cases_obtained,percentage_finished,finished


# Exporting

In [22]:
path = '../Progress reports/'
df2.drop(columns=['_merge']).to_csv(path+'progress_master data tables.csv', index=False, encoding='latin1')